<a href="https://colab.research.google.com/github/saarang2003/Machine-Tranlator/blob/main/Machine%20Language%20Translator%20.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git


In [ ]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [ ]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransToolkit.git
%cd IndicTransToolkit
!python3 -m pip install --editable ./
%cd ..

In [ ]:
%%writefile app.py
import streamlit as st
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer
from IndicTransToolkit import IndicProcessor

# Configuration
BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None  # Options: None, "4-bit", or "8-bit"

# Function to initialize model and tokenizer
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig is None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()
    return tokenizer, model

def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        with tokenizer.as_target_tokenizer():
            decoded = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        translations += ip.postprocess_batch(decoded, lang=tgt_lang)
        del inputs
        if DEVICE == "cuda":
            torch.cuda.empty_cache()
    return translations

@st.cache_resource(show_spinner=False)
def load_model():
    ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"
    tokenizer, model = initialize_model_and_tokenizer(ckpt_dir, quantization)
    ip = IndicProcessor(inference=True)
    return tokenizer, model, ip

def main():
    st.title("IndicTrans2 English-to-Indic Translation")
    st.write("Enter English sentences and get translations in your desired Indic language.")

    st.sidebar.header("Translation Settings")
    src_lang = st.sidebar.selectbox("Source Language", options=["eng_Latn"], index=0)
    target_options = {
        "Assamese": "asm_Beng",
        "Bengali": "ben_Beng",
        "Gujarati": "guj_Gujr",
        "Hindi": "hin_Deva",
        "Kannada": "kan_Knda",
        "Malayalam": "mal_Mlym",
        "Marathi": "mar_Deva",
        "Nepali": "npi_Deva",
        "Oriya": "ori_Orya",
        "Punjabi": "pan_Guru",
        "Tamil": "tam_Taml",
        "Telugu": "tel_Telu",
        "Urdu": "urd_Arab",
    }
    tgt_lang_name = st.sidebar.selectbox("Target Language", list(target_options.keys()))
    tgt_lang = target_options[tgt_lang_name]

    input_text = st.text_area("Enter English sentences (one per line)", height=200)
    if st.button("Translate"):
        if not input_text.strip():
            st.warning("Please enter some text to translate.")
        else:
            input_sentences = [line.strip() for line in input_text.split("\n") if line.strip()]
            with st.spinner("Loading model and translating..."):
                tokenizer, model, ip = load_model()
                translations = batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip)
            st.subheader("Translations")
            for src, tgt in zip(input_sentences, translations):
                st.markdown(f"**English:** {src}")
                st.markdown(f"**{tgt_lang_name}:** {tgt}")
                st.write("---")

if __name__ == "__main__":
    main()


Writing app.py


In [ ]:
!streamlit run app.py &>/dev/null&


In [ ]:
!pip install streamlit pyngrok

In [ ]:
!pip install --upgrade pyngrok


In [ ]:
ngrok.kill()

In [ ]:
from pyngrok import ngrok

# Set your ngrok authtoken (replace with your actual token)
ngrok.set_auth_token("2saHaLzNEOiPkOpNnWLBRe0P6KZ_efXapqC6vMMNp9iKRrR5")

# Optionally, kill any existing tunnels
# ngrok.kill()

# Create a tunnel to the Streamlit port (8501 by default)
public_url = ngrok.connect(8501, "http", bind_tls=False)
print(f"Streamlit app is available at: {public_url}")


Streamlit app is available at: NgrokTunnel: "http://1c27-35-196-201-177.ngrok-free.app" -> "http://localhost:8501"


In [ ]:
%%bash --bg
streamlit run app.py


In [ ]:
!streamlit run app.py &>/dev/null&
